In [113]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

input_path = '../data/stock_data/'
output_path = '../data/output/'

def table1(input_path,holding_days=5):
    # 读取数据
    # stock = 'sh600000'
    # set path to 'data/balabala' if you run Python file
    # input_path = '../data/stock_data/sh600000.csv'
    # df_input = pd.read_csv(input_path, skiprows=1, encoding='GB18030')
    df_input = pd.read_csv(input_path, skiprows=1, encoding='GBK')


    stock_trade_record = pd.DataFrame(columns=['股票代码', '股票名称', '持有期限', '买入日期','卖出日期', '买入价格', '卖出价格','收益率', '浮盈', '浮亏'])


    # 数据初始化
    peak_value = df_input['总市值'][0]
    first_date = datetime.datetime.strptime(df_input['交易日期'][0], '%Y-%m-%d')
    stock_holding = False  # 判断是否持有股票
    return_rate = 1.0  # 收益率
    max_return_rate = 1.0  # 浮亏
    min_return_rate = 1.0  # 浮亏
    win = 0  # 亏利次数
    loss = 0  # 亏损次数
    trade_times = 0  # 交易次数
    # holding_days = 5  # 持有期限

    # 遍历数据，找到股票价值达到新高的时刻
    for i in range(len(df_input)):
        current_day = i
        # current_date = datetime.datetime.strptime(df_input['交易日期'][i], '%Y-%m-%d')
        # 上市日期满250天时，开始判断股票是否符合购买要求
        if i > 250:
            if stock_holding == True:
                # 单日涨跌幅
                change = (df_input['收盘价'][i]/df_input['前收盘价'][i]) - 1

                # 累计收率率
                return_rate *= (1 + change)
                
                # 最大浮亏
                if return_rate > max_return_rate:
                    max_return_rate = return_rate
                # 最大浮亏
                if return_rate < min_return_rate:
                    min_return_rate = return_rate

                # 判断股票的持有期限，此处为5天
                # 持有时间超过期限时，计算指标数据
                # if (current_date - buy_date).days > holding_days:
                if current_day - buy_day > holding_days:

                    # 当股票涨停或跌停时，无法对股票进行操作
                    if df_input['开盘价'][i] != df_input['最高价'][i] and \
                            df_input['最高价'][i] != df_input['最低价'][i] and \
                            df_input['开盘价'][i] != df_input['最低价'][i]:
                        
                        stock_holding = False

                        sell_date = datetime.datetime.strptime(df_input['交易日期'][i], '%Y-%m-%d')
                        sell_value = df_input['开盘价'][i]
                        if return_rate >= 1:
                            win += 1
                        else:
                            loss += 1

                        stock_trade_record.loc[trade_times] = [df_input['股票代码'][i], df_input['股票名称'][i], holding_days,
                                                            buy_date, sell_date, buy_value, sell_value, return_rate-1,
                                                            max_return_rate-1, min_return_rate-1]
                        trade_times += 1
                        return_rate = 1.0
                        max_return_rate = 1.0
                        min_return_rate = 1.0

            # 股票市值达到新高时，认为股票价值达到新高，买入股票并持有
            if df_input['总市值'][i] > peak_value :
                peak_value = df_input['总市值'][i]
                if stock_holding == False:
                    stock_holding = True
                    buy_day = i
                    buy_date = datetime.datetime.strptime(df_input['交易日期'][i], '%Y-%m-%d')
                    buy_value = df_input['开盘价'][i]
        elif i == 250:
            peak_value = df_input['总市值'][:i].max()

    return stock_trade_record,[df_input['股票代码'][i], df_input['股票名称'][i], holding_days]

def table2(table1, table1_info):
    s = table1
    if len(s) > 0:
        t2 = s[['股票代码', '股票名称', '持有期限']].loc[0]
        t2['胜率'] = (s['收益率']>0).sum() / len(s)
        t2['平均收益'] = s['收益率'].mean()
        t2['最大收益'] = s['收益率'].max()
        t2['最小收益'] = s['收益率'].min()
        t2['平均浮盈'] = s['浮盈'].mean()
        t2['最大浮盈'] = s['浮盈'].max()
        t2['最小浮盈'] = s['浮盈'].min()
        t2['平均浮亏'] = s['浮亏'].mean()
        t2['最大浮亏'] = s['浮亏'].min()
        t2['最小浮亏'] = s['浮亏'].max()
        t2['交易次数'] = len(s)
    else:
        t2 = pd.DataFrame(np.array([table1_info[0],table1_info[1],table1_info[2], np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, 0]).reshape(1,-1),
        columns=['股票代码', '股票名称', '持有期限', '胜率', '平均收益', '最大收益', '最小收益', '平均浮盈', '最大浮盈',
        '最小浮盈', '平均浮亏', '最大浮亏', '最小浮亏','交易次数'])
    return t2


file_list = os.listdir(input_path)
csv_list = []
df_output = pd.DataFrame(columns=['股票代码', '股票名称', '持有期限', '胜率', '平均收益', '最大收益', '最小收益', '平均浮盈', '最大浮盈',
        '最小浮盈', '平均浮亏', '最大浮亏', '最小浮亏','交易次数'])

for names in file_list:
    if names.endswith(".csv"):
        csv_list.append(names)

for holding_days in [5,10,15,20,25,30,35,40]:
    for file_name in csv_list[:100]:
        file_path = os.path.join(input_path,file_name)
        t1, t1_info = table1(file_path,holding_days=holding_days)

        t2 = table2(t1,t1_info)
        df_output = df_output.append(t2)
df_output[['持有期限', '胜率', '平均收益', '最大收益', '最小收益', '平均浮盈', '最大浮盈',\
        '最小浮盈', '平均浮亏', '最大浮亏', '最小浮亏','交易次数']] = df_output[['持有期限', '胜率', '平均收益', '最大收益', '最小收益', '平均浮盈', '最大浮盈',\
        '最小浮盈', '平均浮亏', '最大浮亏', '最小浮亏','交易次数']].apply(pd.to_numeric,errors='coerce')

In [103]:

# df_output[[]]
df_group_stock = df_output.groupby(['股票代码'])
for name, group in df_group_stock:
    group.to_excel(output_path + 'groupByStock/' + str(name) + '.xlsx',index=False,)
df_group_duration = df_output.groupby(['持有期限'])
for name, group in df_group_duration:
    group.to_excel(output_path + 'groupByDuration/' + str(name) + '.xlsx',index=False)

In [112]:
df_output

,股票代码,股票名称,持有期限,胜率,平均收益,最大收益,最小收益,平均浮盈,最大浮盈,最小浮盈,平均浮亏,最大浮亏,最小浮亏,交易次数
0,sh600632,华联商厦,5,0.0,-0.133746,-0.133746,-0.133746,0.20805,0.20805,0.20805,-0.133746,-0.133746,-0.133746,1
0,sh600632,华联商厦,10,0.0,-0.114551,-0.114551,-0.114551,0.20805,0.20805,0.20805,-0.159133,-0.159133,-0.159133,1
0,sh600632,华联商厦,15,0.0,-0.176471,-0.176471,-0.176471,0.20805,0.20805,0.20805,-0.176471,-0.176471,-0.176471,1
0,sh600632,华联商厦,20,0.0,-0.195046,-0.195046,-0.195046,0.20805,0.20805,0.20805,-0.202477,-0.202477,-0.202477,1
0,sh600632,华联商厦,25,0.0,-0.241486,-0.241486,-0.241486,0.20805,0.20805,0.20805,-0.286068,-0.286068,-0.286068,1
0,sh600632,华联商厦,30,0.0,-0.214241,-0.214241,-0.214241,0.20805,0.20805,0.20805,-0.286068,-0.286068,-0.286068,1
0,sh600632,华联商厦,35,0.0,-0.204334,-0.204334,-0.204334,0.20805,0.20805,0.20805,-0.286068,-0.286068,-0.286068,1
0,sh600632,华联商厦,40,0.0,-0.201238,-0.201238,-0.201238,0.20805,0.20805,0.20805,-0.286068,-0.286068,-0.286068,1


In [97]:
tmp = pd.DataFrame([[1,'nan'],[3,2],[4,5]],columns=['a','b'])

tmp.dtypes

a     int64
b    object
dtype: object

In [99]:
tmp[['a','b']]=tmp[['a','b']].apply(pd.to_numeric,errors='coerce')
tmp.dtypes

a      int64
b    float64
dtype: object

In [100]:
tmp

,a,b
0,1,NaN
1,3,2.0
2,4,5.0
